# Ag-Analytics® - Farmland Sales API 

Ag-Analytics® Farmland Sales API provides users with easy and fast access to filter and find land for sale, complete with the sales and geographic data components to get a detailed description of that land’s value. The data of this service is originally provided by [Farmland Finder](https://www.farmlandfinder.com). In this version of Land Value Service API, each parcel/land has a full description of the entire sale/transaction. Multiple parcel/land record can belong to one transaction. In geographic attribute, each result record gives the detailed description of a certain land/parcel itself, while the attribute in ‘sales’ may carry the information for the transaction/sale other than this parcel/land.

### Required libraries

In [81]:
import requests
import json
import pandas as pd
import folium
import mplleaflet
import geojson
import shapely.wkt
import os

### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/farmland-sales/

       
1). __Location Parameters__(Required): 
    To make a valid request,at least one location parameters need to be provided.
    
    i. State:  The name of State in string format.Title cased.
    
    ii. County: The name of State in string format. Title cased.This parameter will be valid only if the State parameter is provided
    
    iii.Bounding_box: Area of interest in geoJSON format(See example below).
    

2). __Sale Condition Parameters__(optional): Optional parameters to specify response based on sale conditions

    i. Status: Sale Condition of the property. Vaild options are: Sold, For Sale, Expired Listing.
    
    ii. StartDate (Required only if Status is Sold):Searching starting date of the property Sale Date. In format 'yyyy-mm-dd'
    
    iii.EndDate (Required only if Status is Sold):Searching end date of the property Sale Date. In format 'yyyy-mm-dd'

### Response Description

__Listing Information__

1).Listing_id: The unique ID for each listing(transaction). 

2).Entry_Updated:The date of the sales information has been updated

3).Avg_CSR2: the average The Iowa Corn Suitability Rating(Soil Productivity Index)

4).CRP: If the property joined the Conservation Reserve Program.(‘Yes’ or ‘No’)

5).Total_Acres: the total acres of the entire sale

6).Tillable_Acres: the tillable acres of the sale

7).Percent_Tillable: the percent tillable area

8).CRP_Acres: the CRP acres

9).Sale_Price: the total sale price of the sale record

10).Price_Acre: the price per acre of the sale record

11).Status: one of the following values: "For Sale", "Listing Expired", "Sold"

12).Sale_Condition: one of the following values: "Auction", "Listing"

13).Listing_Agent: the listing agent name

14).Buyer: the buyer name as a string

15).Sale_Date: The sale date string in YYYY-MM-DD format. (When the parcel is still listing, the attribute will be Null

16).Taxes_total: taxes for the sale as a float

17).Assessed_Land: If the land/parcel has been assessed

18).Broker_URL: The URL link to the broker listing webpage as a string


__Parcel Information__

1).Parcel_ID :The unique sale id for each parcel as a string

2).Parcel: Index of the parcel in one listing(transaction).

3).Shape: The boundary of the parcel/property in Well Known Text type

4).GeoJSON: The boundary of the parcel/property in GeoJSON type

5).Acres: Area of the parcel

6).State: The state where the parcel/property locates in. 

7).County:The county where the parcel/property locates in.

8).lat_center: the latitudinal center of the parcel/property as a float value

9).lng_center: the longitudinal center of the parcel/property as a float value

10).range: the range as a string (ex: 26W - always include E or W)

11).sect: the section as string (ex: 17)

12).twnshp: the township as a string (ex: 78N - always include N or S)

13).county_name: the county name as a string (Title cased - for ex: Osceola, Polk, etc.)

14).state_name: the state name as a string (Title cased - for ex: Michigan)

15).STATEFP: the FIPS to state level as a string

16).FIPS: the FIPS to county level as a string


### Request Parameter Example

In [134]:
State= "Michigan"
County= "Houghton"
Bounding_box = '{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-88.95973205566405,46.68995749641134],[-88.76678466796875,46.68995749641134],[-88.76678466796875,46.7981792512332],[-88.95973205566405,46.7981792512332],[-88.95973205566405,46.68995749641134]]]}}'

Status = 'Sold'
StartDate = '2019-08-08'
EndDate = '2020-01-01'

#Format the inputs()
values = {
    'State':State,
    'County' : County,
    'Bounding_Box': Bounding_box,
    #'StartDate':StartDate,
    #'EndDate':EndDate,
    #'Status' :Status
}

# Header for using a subscription key.
headers = {'Content-Type': 'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': "XXXXXXXXXXXXXX"}

### API Function

In [143]:
def LandValueService(values,headers):
    try:
        url = 'https://ag-analytics.azure-api.net/farmland-sales/'
        
        response = requests.post(url, data = values,headers = headers).json()
        print(response.get('status'))
        print(response.get('msg'))
        print(response)
        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [144]:
LandValueResponse = LandValueService(values,headers)

SUCCESS
3 parcel records
{'msg': '3 parcel records', 'records': {'09A7DDA8-2B75-4DAF-BA05-BE5FEE0747A9': {'Assessed_Land': None, 'Avg_CSR2': 21.76897, 'Broker_URL': 'http://www.northernmichiganlandbrokers.com/pages/listings/details.php?mls=1112205', 'Buyer': None, 'CRP': 'No', 'CRP_Acres': None, 'Entry_Updated': 'Fri, 15 Mar 19 00:00:00 GMT', 'Listing_Agent': 'Northern Michigan Land Brokers', 'Listing_id': '09A7DDA8-2B75-4DAF-BA05-BE5FEE0747A9', 'Percent_Tillable': 1.0, 'Price_Acre': 809.0, 'Sale_Condition': 'Listing', 'Sale_Date': None, 'Sale_Price': 129500.0, 'Status': 'For Sale', 'Taxes_total': None, 'Tillable_Acres': 1.3, 'Total_Acres': 160.0, 'parcels': [{'Acres': 160.01292273964387, 'County': 'Houghton', 'FIPS': '26061', 'GeoJSON': {'coordinates': [[[-88.8189026265523, 46.7153441141984], [-88.8241335031313, 46.7153320080954], [-88.8241452523484, 46.7116995060643], [-88.8241569999999, 46.708067], [-88.8188885071246, 46.708091861304], [-88.8136200094686, 46.7081164817389], [-88.808

### Format Response

In [145]:
listing_info =[]
records = LandValueResponse.get('records')
for listing in records:
    parcels_info = records.get(listing).get('parcels')
    list_item = records.get(listing)
    list_item.pop('parcels')
    for parcel in parcels_info:
        parcel.update(list_item)
        listing_info.append(parcel)

listing_db = pd.DataFrame(listing_info)

### Display Response

In [146]:
#Narrow table
listing_db.set_index('Listing_id').T
#Wide table
# listing_db.set_index('Listing_id')

Listing_id,09A7DDA8-2B75-4DAF-BA05-BE5FEE0747A9,F7B64E2F-B4F2-45CC-98E5-2007C3D3A48D,F7B64E2F-B4F2-45CC-98E5-2007C3D3A48D
Acres,160.013,234.654,80.8075
Assessed_Land,None,None,None
Avg_CSR2,21.769,22.3867,22.3867
Broker_URL,http://www.northernmichiganlandbrokers.com/pag...,http://www.northernmichiganlandbrokers.com/pag...,http://www.northernmichiganlandbrokers.com/pag...
Buyer,None,None,None
CRP,No,No,No
CRP_Acres,None,None,None
County,Houghton,Houghton,Houghton
Entry_Updated,"Fri, 15 Mar 19 00:00:00 GMT","Mon, 14 Oct 19 00:00:00 GMT","Mon, 14 Oct 19 00:00:00 GMT"
FIPS,26061,26061,26061


### Visualize Parcels on Map

In [147]:
#Display parcels on map
zoom_point=json.loads(Bounding_box)['geometry']['coordinates'][0][0]
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in listing_db['GeoJSON']:
    folium.GeoJson(shape).add_to(m)
m

### Download Response

In [83]:
download_path = '<---path on your local directory--->' # ex:r'C:\Users\YourName\Downloads'
listing_db.to_csv(os.path.join(download_path,'FarmLand_Listing.csv'))